# 🧪 GraphSDMining: Subgroup Discovery Example
This notebook demonstrates how to load position and social data, bin attributes, construct an interaction graph, mine subgroups, and visualize the top pattern.

In [ ]:
import logging
from graphsd import GraphSDMining
from graphsd.datasets import load_data
from graphsd.utils import make_bins
from graphsd.viz import graph_viz

# Enable logging
logging.basicConfig(level=logging.INFO)


## Step 1: Load Dataset

In [ ]:
position_df, social_df = load_data("playground_a")
position_df.head(), social_df.head()


## Step 2: Discretize Social Attributes

In [ ]:
social_df = make_bins(social_df, n_bins=3, strategy="quantile")
social_df.head()


## Step 3: Create and Configure GraphSDMining Model

In [ ]:
model = GraphSDMining(random_state=42, n_jobs=1)


## Step 4: Read Data and Construct Graph

In [ ]:
model.read_data(position_df, social_df, time_step=10, proximity=1.0)


## Step 5: Discover Patterns

In [ ]:
patterns = model.subgroup_discovery(mode="comparison", min_support=0.2, metric="mean", quality_measure="qP")


## Step 6: View Pattern Summary

In [ ]:
df = model.to_dataframe(patterns)
df.head()


## Step 7: Visualize Top Pattern

In [ ]:
if patterns:
    graph_viz(patterns[0].graph, title=str(patterns[0].name))
